In [ ]:
%load_ext autoreload
%autoreload 2
import torch

from plstm.torch.plstm_1d import pLSTM1D_fwbw, pLSTM1D_torch
from plstm.torch.test_utils import check_forward, check_backward
from plstm.torch.util import log2

In [ ]:
def normalize(x, dim=0):
    y = x - torch.mean(x, dim=dim, keepdim=True)
    return y / torch.sqrt(torch.mean(y**2, dim=dim, keepdim=True))

In [ ]:
normalize(torch.arange(5.0)[None, :], dim=1)

In [ ]:
B = 1
T = 32
BT = 8
DHQK = 11
DHHV = 5
JQ = 1
JT = 1
JV = 1
JK = 1
JO = 1
DTYPE = torch.float64

rand_factor = 0.0
Q = 1.0 + rand_factor * torch.randn([B, T, DHQK, JQ], dtype=DTYPE) / DHQK
K = 1.0 + rand_factor * torch.randn([B, T, DHQK, JK], dtype=DTYPE)
V = torch.randn([B, T, DHHV, JV], dtype=DTYPE) + 0.1 * torch.arange(JV * DHHV).reshape(1, 1, DHHV, JV)
S0 = 0.1 + rand_factor * torch.randn([B, T, JT, JK, JV], dtype=DTYPE)
T0 = torch.eye(JT)[None, None, :, :] * torch.ones([B, T, 1, 1], dtype=DTYPE)
T0 = T0 + 0.01 * rand_factor * torch.randn_like(T0)
M0 = 1.0 + rand_factor * torch.randn([B, T, JO, JQ, JT], dtype=DTYPE)
D0 = 1.0 + rand_factor * torch.randn([B, T, JO, JQ, JK, JV], dtype=DTYPE)
C_initial = 0.0 * torch.randn([B, DHQK, DHHV, JT], dtype=DTYPE)

print(Q.shape)

Q.requires_grad_(True)
K.requires_grad_(True)
V.requires_grad_(True)
S0.requires_grad_(True)
T0.requires_grad_(True)
M0.requires_grad_(True)

S0mag = (
    0.01
    # + 0.1* rand_factor * torch.randn([B, T])
    - 0.1 * torch.arange(T)[None, :]
)
T0mag = (
    -0.01 + 0.0 * 0.1 * rand_factor * 0.01 * torch.randn([B, T])
    # + 0.01 * torch.arange(T)[None, :]
)

_ = ""
# Y = pLSTM1D_fwbw(Q, K, V, S0, M0, T0, chunk_size=16)
# Y = Y.reshape(B, T, DHHV * JQ)[0, :, 0]

# Y.sum().backward()

In [ ]:
check_forward(
    lambda q, k, v, s0, t0, m0, d0, c0: normalize(
        pLSTM1D_torch(q, k, v, s0, t0, m0, d0, C_initial=c0, levels=log2(BT)),
        dim=2,
    ).flatten(2, 3),
    lambda q, k, v, s0, t0, m0, d0, c0: normalize(
        pLSTM1D_torch(q, k, v, s0, t0, m0, d0, C_initial=c0, levels=log2(BT // 2)),
        dim=2,
    ).flatten(2, 3),
    (Q, K, V, S0, T0, M0, D0, C_initial),
    verbose=True,
)

In [ ]:
# check_forward(
#     lambda q, k, v, s0, t0, m0, d0, c0: pLSTM1D_torch(
#         q, k, v, s0, t0, m0, d0, C_initial=c0, levels=log2(BT)
#     ),
#     lambda q, k, v, s0, t0, m0, d0, c0: pLSTM1D_fwbw(
#         q, k, v, s0, t0, m0, d0, S0mag, T0mag, C_initial=c0, levels=log2(BT)
#     ),
#     (Q, K, V, S0, T0, M0, D0, C_initial),
#     verbose=True,
# )

In [ ]:
check_forward(
    lambda q, k, v, s0, t0, m0, d0, s0m, t0m, c0: normalize(
        pLSTM1D_torch(q, k, v, s0, t0, m0, d0, s0m, t0m, C_initial=c0, levels=log2(BT)),
        dim=2,
    ).flatten(2, 3),
    lambda q, k, v, s0, t0, m0, d0, s0m, t0m, c0: normalize(
        pLSTM1D_torch(q, k, v, s0, t0, m0, d0, s0m, t0m, C_initial=c0, levels=log2(BT // 2)),
        dim=2,
    ).flatten(2, 3),
    (Q, K, V, S0, T0, M0, D0, S0mag, T0mag, C_initial),
    verbose=True,
    atol=1e-3,
    rtol=1e-3,
)

In [ ]:
check_forward(
    lambda q, k, v, s0, t0, m0, d0, s0m, t0m, c0: normalize(
        pLSTM1D_torch(q, k, v, s0, t0, m0, d0, s0m, t0m, C_initial=c0, levels=log2(BT)),
        dim=2,
    ).flatten(2, 3),
    lambda q, k, v, s0, t0, m0, d0, s0m, t0m, c0: normalize(
        pLSTM1D_fwbw(q, k, v, s0, t0, m0, d0, s0m, t0m, C_initial=c0, levels=log2(BT // 2)),
        dim=2,
    ).flatten(2, 3),
    (Q, K, V, S0, T0, M0, D0, S0mag, T0mag, C_initial),
    verbose=True,
    atol=1e-3,
    rtol=1e-3,
)

In [ ]:
torch.autograd.set_detect_anomaly(True)
a = pLSTM1D_torch(
    Q,
    K,
    V,
    S0,
    T0,
    M0,
    D0,
    S0mag=S0mag,
    T0mag=T0mag,
    C_initial=C_initial,
    levels=log2(BT),
)
a.sum().backward(retain_graph=True)
a.shape

In [ ]:
return_last_C = False
use_initial_C = True
check_backward(
    lambda q, k, v, s0, t0, m0, d0, c0: normalize(
        pLSTM1D_torch(
            q,
            k,
            v,
            s0,
            t0,
            m0,
            d0,
            C_initial=c0,
            levels=log2(BT),
            return_last_C=return_last_C,
        ).flatten(2, 3),
        dim=2,
    ),
    lambda q, k, v, s0, t0, m0, d0, c0: normalize(
        pLSTM1D_fwbw(
            q,
            k,
            v,
            s0,
            t0,
            m0,
            d0,
            C_initial=c0,
            levels=log2(BT),
            recompute_C=True,
            recompute_G=True,
            use_initial_C=use_initial_C,
            return_last_C=return_last_C,
        ).flatten(2, 3),
        dim=2,
    ),
    (Q, K, V, S0, T0, M0, D0, C_initial if use_initial_C else None),
    verbose=True,
    rand_factor=1.0,
    atol=1e-4,
    rtol=1e-2,
)

In [ ]:
return_last_C = False
use_initial_C = True
check_backward(
    lambda q, k, v, s0, t0, m0, d0, sm, tm, c0: normalize(
        pLSTM1D_torch(
            q,
            k,
            v,
            s0,
            t0,
            m0,
            d0,
            sm,
            tm,
            C_initial=c0,
            levels=log2(BT) // 2,
            return_last_C=return_last_C,
        ),
        dim=2,
    ).flatten(2, 3),
    lambda q, k, v, s0, t0, m0, d0, sm, tm, c0: normalize(
        pLSTM1D_fwbw(
            q,
            k,
            v,
            s0,
            t0,
            m0,
            d0,
            sm,
            tm,
            C_initial=c0,
            levels=log2(BT),
            use_initial_C=use_initial_C,
            # recompute_C=False,
            # recompute_G=False,
            return_last_C=return_last_C,
        ),
        dim=2,
    ).flatten(2, 3),
    (Q, K, V, S0, T0, M0, D0, S0mag, T0mag, C_initial if use_initial_C else None),
    verbose=True,
    rand_factor=1.0,
    atol=1e-4,
    rtol=5e-2,
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from plstm.torch.plstm_1d_layer import pLSTM1DLayer, pLSTM1DLayerConfig

In [ ]:
D = 512
H = 4
JK = JQ = 1
JV = JO = JT = 4

plstm = (
    pLSTM1DLayer(
        pLSTM1DLayerConfig(
            DK=D // JK,
            num_heads=H,
            DV=D // JV,
            JQ=JQ,
            JK=JK,
            JV=JV,
            JT=JT,
            JO=JO,
            input_dim=D,
            sub_heads=16,
        )
    )
    .to(device=device)
    .to(dtype=torch.float16)
)

In [ ]:
B, T = 3, 2048

o = plstm(torch.randn([B, T, H * D], device=device, dtype=torch.float16) * D ** (-1 / 2))
print(o.shape)
print(torch.max(o))

In [ ]:
o = plstm(torch.randn([B, T, H * D], device=device, dtype=torch.float16) * D ** (-1 / 2))

In [ ]:
o.sum().backward()

In [ ]:
plstm_tc = torch.compile(plstm)

In [ ]:
o2 = plstm_tc(torch.randn([B, T, H * D], device=device, dtype=torch.float16) * D ** (-1 / 2))

In [ ]:
o3 = plstm_tc(torch.randn([B, T, H * D], device=device, dtype=torch.float16) * D ** (-1 / 2))